In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 6.9 MB/s eta 0:00:00


In [4]:
import torch
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from optimizers_impl.sam.sam import SAM
from utility.bypass_bn import enable_running_stats, disable_running_stats

In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)
        enable_running_stats(model)

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        disable_running_stats(model)
        smooth_crossentropy(model(inputs), targets, smoothing=0.1).mean().backward()
        optimizer.second_step(zero_grad=True)

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())


In [8]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'widerestnet-sam'
    model_name = model_name_path + model_name
    initialize(seed=22)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)

    base_optimizer = torch.optim.SGD
    optimizer = SAM(model.parameters(), base_optimizer, rho=0.1, adaptive=True, lr=0.0003, weight_decay=0.6)

    scheduler = StepLR(optimizer, 0.027, num_epochs)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3474  │      0.61 %  ┃   2.700e-02  │       00:17  ┃      0.3681  │     51.00 %  ┃
┃           1  ┃      0.3086  │      0.77 %  ┃   2.700e-02  │       00:03  ┃      0.3676  │     50.00 %  ┃
┃           2  ┃      0.2741  │      0.80 %  ┃   2.700e-02  │       00:03  ┃      0.3677  │     50.00 %  ┃
┃           3  ┃      0.2405  │      0.83 %  ┃   2.700e-02  │       00:03  ┃      0.3660  │     50.00 %  ┃
┃           4  ┃      0.2091  │      0.86 %  ┃   2.700e-02  │       00:03  ┃      0.3649  │     50.00 %  ┃
┃           5  ┃      0.1874  │      0.87 %  ┃   2.700e-02  │       00:03  ┃      0.3649  │     50.00 %  ┃
┃           6  ┃      0.1740  │      0.88 %  ┃   2.700e-02  │       00:03  ┃      0.3681  │     50.00 %  ┃
┃           7  ┃      0.1684  │      0.88 %  ┃   2.700e-02  │       00:03  ┃      0.3680  │     50.00 %  ┃
┃           8  ┃      0.1672  │      0.88 %  ┃   2.700e-02  │       00:03  ┃      0.3675  │     50.00 %  ┃
┃           9  ┃      0.1624  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)

tensor(0.9615, device='cuda:0')
tensor(0.9600, device='cuda:0')
tensor(0.9259, device='cuda:0')
tensor(0.9200, device='cuda:0')
tensor(1., device='cuda:0')
